# Custom LLLM Agent

This notebook goes through how to create your own custom LLM agent.

An LLM agent consists of three parts:
    
    - Tools: The tools the agent has available to use.
    - LLMChain: The LLMChain that produces the text that is parsed in a certain way to determine which action to take.
    - OutputParser: This determines how to parse the LLMOutput into 
        
        
In this notebook we walk through how to create a custom LLM agent.

In [1]:
from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent import LLMAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [3]:
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Search: useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        return template.format(**kwargs)

prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps"])

FINAL_ANSWER_ACTION = "Final Answer:"
from langchain.schema import AgentAction, AgentFinish
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output):
        if FINAL_ANSWER_ACTION in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split(FINAL_ANSWER_ACTION)[-1].strip()},
                log=llm_output,
            )
        # \s matches against tab/newline/whitespace
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [8]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [12]:
tool_names = [tool.name for tool in tools]
agent = LLMAgent(llm_chain=llm_chain, output_parser=output_parser, stop=["\nObservation:"], allowed_tools=tool_names)

In [13]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Canada in 2023
Action: Search
Action Input: Population of Canada in 2023

Observation:The current population of Canada is 38,644,767 as of Tuesday, March 28, 2023, based on Worldometer elaboration of the latest United Nations data. I now know the final answer
Final Answer: Arrr, Canada be havin' 38,644,767 people livin' there as of 2023!

> Finished chain.


"Arrr, Canada be havin' 38,644,767 people livin' there as of 2023!"